In [2]:
from sgp4.earth_gravity import wgs72
from sgp4.io import twoline2rv
from skyfield.sgp4lib import TEME_to_ITRF
from skyfield.api import load as skyfield_load
import datetime
import pymap3d
import numpy as np
from shapely import geometry
import geopandas as gpd

In [3]:
G1_TLE = ["1 43730U 18096M   19263.77135048  .00000515  00000-0  22250-4 0  9993",
          "2 43730  97.4553 333.8276 0020037   9.3158 350.8449 15.26869367 45108"]
G2_TLE = ["1 43812U 18099BG  19264.16418962  .00000060  00000-0  10634-4 0  9998",
          "2 43812  97.7354 334.9858 0014097  21.3762 338.8045 14.95420297 43132"]
G3_TLE = ["1 44367U 19037C   19263.24019284  .00001613  00000-0  51514-4 0  9996",
          "2 44367  45.0113 188.9917 0009622  34.2602 325.8905 15.37721792 12783"]
G4_TLE = ["1 44499U 19054E   19264.20700450 -.00000104  00000-0  91357-5 0  9995",
          "2 44499  45.0168 282.3238 0007537  97.4852 305.3036 15.07651419  4925"]

In [4]:
G2 = twoline2rv(G2_TLE[0], G2_TLE[1], wgs72)

In [5]:
epoch = datetime.datetime(2019,9,23)
delta = datetime.timedelta(minutes=1)

time_arr = [epoch + n*delta for n in range(1440)]

ts = skyfield_load.timescale()
epoch_ut1 = ts.ut1(epoch.year, epoch.month, epoch.day, epoch.hour, epoch.minute, epoch.second)

In [7]:
pos_teme = []
vel_teme = []
for time in time_arr:
    pos, vel = G2.propagate(time.year, time.month, time.day, time.hour, time.minute, time.second)
    
    pos_teme.append(pos)
    vel_teme.append(vel)
    
pos_teme = np.array(pos_teme)
vel_teme = np.array(vel_teme)

In [8]:
pos_icrf, vel_icrf = TEME_to_ITRF(epoch_ut1.tt, pos_teme.transpose(), vel_teme.transpose()*86400)

In [10]:
pos_ecef = []

for idx in range(1440):
    eci = pos_icrf[:,idx].reshape(1,3)
    t = time_arr[idx]
    
    ecef = pymap3d.eci2ecef(eci * 1000, t)
    pos_ecef.append(ecef)
    
pos_ecef = np.array(pos_ecef)

In [11]:
lat, lon, alt = pymap3d.ecef2geodetic(pos_ecef[:,0], pos_ecef[:,1], pos_ecef[:,2])

In [12]:
lon_lat = np.concatenate([lon, lat], axis = 1)

In [13]:
s = geometry.LineString(lon_lat)

In [17]:
geo = gpd.GeoDataFrame({"col": [20]}, geometry=[s])

In [19]:
geo.to_file("test.geojson",driver="GeoJSON")